<a href="https://colab.research.google.com/github/MudSnail/Land_Cover_Classification/blob/main/Basic_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import base libraries
import pandas as pd
import numpy as np
import copy
import joblib
import cloudpickle

#Sklearn - Pipelines, Training
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

#Classification Models
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#Model metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, roc_auc_score, precision_score, recall_score

#Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#misc
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Input Data, Combine and Select

In [3]:
#Read in CSVs
james_bay = pd.read_csv('/content/drive/MyDrive/james_bay_data.csv')
labrador = pd.read_csv('/content/drive/MyDrive/labrador_data.csv')
simcoe = pd.read_csv('/content/drive/MyDrive/simcoe_york_data.csv')

In [4]:
#List dataframes to add to james_bay
dfs = [labrador, simcoe]

#Merged data
merged_data = james_bay.append(dfs)

#Print shape and check new data
print(merged_data.shape)
merged_data.head()

(8917500, 13)


,B01,B06,B11,B07,B05,B03,B09,B12,B8A,B04,B08,B02,Class
0,0.0055,0.0017,0.0009,0.0016,0.0026,0.0056,0.0001,0.0009,0.0003,0.0032,0.0012,0.0114,0
1,0.0054,0.0020,0.0009,0.0017,0.0021,0.0060,0.0001,0.0005,0.0007,0.0028,0.0014,0.0115,0
2,0.0058,0.0018,0.0007,0.0016,0.0028,0.0060,0.0001,0.0008,0.0007,0.0030,0.0012,0.0124,0
3,0.0057,0.0016,0.0007,0.0016,0.0023,0.0058,0.0001,0.0012,0.0006,0.0031,0.0011,0.0119,0
4,0.0066,0.0013,0.0008,0.0020,0.0029,0.0059,0.0001,0.0008,0.0005,0.0030,0.0014,0.0115,0


In [5]:
#Check classifications values
print(merged_data.Class.value_counts())

15    2836714
18    1050988
1      677877
5      666952
16     646870
13     612009
8      575175
17     512930
12     415493
6      378274
2      151560
11     135890
14      94850
10      86338
19      74619
0         961
Name: Class, dtype: int64


In [6]:
#Drop Class = 0 (null values, which equate to 18)
merged_data = merged_data[merged_data.Class != 0]

#Double check value counts
merged_data.Class.value_counts()

15    2836714
18    1050988
1      677877
5      666952
16     646870
13     612009
8      575175
17     512930
12     415493
6      378274
2      151560
11     135890
14      94850
10      86338
19      74619
Name: Class, dtype: int64

In [7]:
#SubSample ~25 000 for each class
sample = merged_data.groupby('Class').apply(lambda x: x.sample(n=25000)).reset_index(drop=True)
sample.head(4)

,B01,B06,B11,B07,B05,B03,B09,B12,B8A,B04,B08,B02,Class
0,0.0111,0.1315,0.1054,0.1491,0.0660,0.0315,0.1687,0.0562,0.1670,0.0248,0.1574,0.0180,1
1,0.0106,0.1237,0.0943,0.1398,0.0623,0.0321,0.1573,0.0506,0.1531,0.0247,0.1443,0.0171,1
2,0.0168,0.1535,0.1180,0.1627,0.0681,0.0324,0.2033,0.0607,0.1906,0.0281,0.1785,0.0176,1
3,0.0177,0.1588,0.1359,0.1783,0.0857,0.0397,0.2038,0.0701,0.1988,0.0348,0.1900,0.0244,1


In [8]:
#Check basic stats
sample.describe()

,B01,B06,B11,B07,B05,B03,B09,B12,B8A,B04,B08,B02,Class
count,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000,375000.000000
mean,0.066077,0.183755,0.183016,0.201909,0.129001,0.089028,0.229438,0.122814,0.223028,0.091400,0.220332,0.072250,11.133333
std,0.131806,0.125040,0.098064,0.122666,0.131654,0.133283,0.126898,0.089094,0.122005,0.134897,0.127692,0.135681,5.512014
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.015600,0.134700,0.126900,0.150900,0.078600,0.039200,0.177800,0.067400,0.170600,0.033400,0.164500,0.020000,6.000000
50%,0.025700,0.166700,0.175900,0.187500,0.102900,0.052700,0.218300,0.103500,0.213100,0.055300,0.206200,0.033900,12.000000
75%,0.051600,0.205000,0.234900,0.231600,0.129500,0.077000,0.262400,0.160000,0.261800,0.088300,0.257600,0.058100,16.000000
max,1.781900,1.757900,1.600300,1.733600,1.782900,1.769900,1.610600,1.593300,1.704300,1.768800,1.745100,1.821400,19.000000


# Split data

In [9]:
#Set x and y
y = sample['Class']
X = sample.drop('Class', axis = 1)

#Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [10]:
#check shape
X_train.shape, y_train.shape

((262500, 12), (262500,))

# Create Pipelines

In [11]:
#Functions for Pipeline
def add_layers(df):
  """
  This function takes in a dataframe and calculates the NDVI, Moisture Index, NDWI and NDSI
  Outputs = dataframe with added layer columns 
  """
  #Create NDVI column (B08-B04)/(B08+B04)
  df['NDVI'] = (df.B08 - df.B04)/(df.B08 + df.B04)
  #Create Moisture index (B8A-B11)/(B8A+B11)
  df['Moisture'] = (df.B8A - df.B11)/(df.B8A + df.B11)
  #Create NDWI (B3-B8)/(B3+B8)
  df['NDWI'] = (df.B03 - df.B08)/(df.B03 + df.B08)
  #create NDSI (B3-B11)/(B3+B11)
  df['NDSI'] = (df.B03 - df.B11)/(df.B03 + df.B11)

  return df

def replace_values(df):
  """
  This function replaces the infinity values with Nan then replaces that with new infinity values
  """
  #Replace infinity values with Nan
  df.replace([np.inf, -np.inf], np.NAN, inplace=True)

  #Fill in null values
  df.fillna(999, inplace=True)

  return df


#Define object transformer class
class objectTransformer:
    """
    Class object transformer, takes in function to apply in preprocessing step of Pipeline
    """
    def __init__(self, func):
        self.func = func
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **fit_params):
        return self.func(X)

## Test PCA and Feature Union

In [12]:
#Create Pre-Processing Pipeline
preprocess = Pipeline([("add_layers", objectTransformer(add_layers)),
                       ("replace_values", objectTransformer(replace_values))])

#Create Feature Union
feature_union = FeatureUnion([("PCA", PCA(n_components=6)),
                              ("kKBest", SelectKBest(k=2))])


In [ ]:
#Define pipelines
model_1 = Pipeline([('preprocessing', preprocess),
                    ('RFC', RandomForestClassifier(random_state = 123))])

model_2 = Pipeline([('preprocessing', preprocess),
                    ('features', feature_union),
                    ('RFC', RandomForestClassifier(random_state = 123))])

model_3 = Pipeline([('preprocessing', preprocess),
                    ('pca', PCA(n_components=6)),
                    ('RFC', RandomForestClassifier(random_state = 123))])

In [ ]:
#fit pipes to check which is best
pipes = [model_1, model_2, model_3]

for pipe in pipes:
  pipe.fit(X_train, y_train)

#compare accuracies 
#make dictionary
pipe_dict = {0: 'No Feature Union', 1: 'With Feature Union', 2: 'Only PCA'}

#create for loop to find scores
for i, model in enumerate(pipes):
    print('{} Test Accuracy: {}'.format(pipe_dict[i], model.score(X_test, y_test)))

No Feature Union Test Accuracy: 0.6150577777777778
With Feature Union Test Accuracy: 0.5676888888888889
Only PCA Test Accuracy: 0.5371822222222222


## Compare SVC, Random Forest, XGBoost

In [13]:
#Compare three model types
svc_pipe = Pipeline([('preprocessing', preprocess),
                    ('SVC', SVC(random_state = 123))])

rfc_pipe = Pipeline([('preprocessing', preprocess),
                    ('RFC', RandomForestClassifier(random_state = 123))])

xgb_pipe = Pipeline([('preprocessing', preprocess),
                    ('XGB', XGBClassifier(random_state = 123))])

In [14]:
#fit pipes to check which is best
pipes = [svc_pipe, rfc_pipe, xgb_pipe]

for pipe in pipes:
  pipe.fit(X_train, y_train)

#compare accuracies 
#make dictionary
pipe_dict = {0: 'SVC', 1: 'Random Forest', 2: 'XGBoost'}

#create for loop to find scores
for i, model in enumerate(pipes):
    print('{} Test Accuracy: {}'.format(pipe_dict[i], model.score(X_test, y_test)))

SVC Test Accuracy: 0.39609777777777777
Random Forest Test Accuracy: 0.6154133333333334
XGBoost Test Accuracy: 0.5536622222222222
